In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

## Demand Data

In [3]:
DemandDF = pd.read_csv('WoolworthsDemand2025.csv', index_col=0)

In [4]:
week_numbers = (np.arange(28) % 7) + 1

SaturdayDF = DemandDF.T.iloc[week_numbers==6].T
WeekdaysDF = DemandDF.T.iloc[week_numbers<6].T

WeekStoreMean = WeekdaysDF.mean(axis=1)
SaturdayMean = SaturdayDF.mean(axis=1)

WeekStoreSD = WeekdaysDF.std(axis=1)
SaturdaySD = SaturdayDF.std(axis=1)

WeekStore75 = WeekStoreMean + 0.675*WeekStoreSD
Saturday75 = SaturdayMean + 0.675*SaturdaySD




In [5]:
demand75 = pd.DataFrame({
    'Weekday': WeekStore75,
    'Saturday': Saturday75,
})

display(demand75)

,Weekday,Saturday
Store,,
FreshChoice Cannons Creek,2.587546,0.000000
FreshChoice Cuba Street,2.246321,0.000000
FreshChoice Woburn,2.194569,0.000000
Metro Cable Car Lane,2.587546,0.000000
Woolworths Aotea,3.034719,1.175000
Woolworths Crofton Downs,3.675902,0.587500
Woolworths Johnsonville,3.852079,0.587500
Woolworths Johnsonville Mall,3.035806,1.396263
Woolworths Karori,2.908907,0.587500


## Routes

In [6]:
TravelDF = pd.read_csv('WoolworthsDurations2025.csv', index_col=0)

In [ ]:
#With API access, we can fetch real traffic data to calibrate our multipliers.
import requests
import numpy as np
from scipy.stats import lognorm
from datetime import datetime, timedelta

# TomTom API key
API_KEY = 'JkaOFQWixewApghm2i0mUHUNXZTTk8QT'

# 3 representative routes for better congestion sampling (lat,lon pairs)
routes = [
    ('-41.2797,174.8035:-41.2923,174.7785', 'CentrePort to Woolworths Newtown (urban, ~5km)'),  # Congested city
    ('-41.2797,174.8035:-41.1333,174.85', 'CentrePort to Woolworths Porirua (suburban, ~15km)'),  # Original
    ('-41.2797,174.8035:-41.086,174.870', 'CentrePort to Woolworths Johnsonville (highway, ~10km)')  # SH1 mix
]
base_url = 'https://api.tomtom.com/routing/1/calculateRoute/{}json'

def get_multiplier(depart_at_utc, route_coords):
    url = f'https://api.tomtom.com/routing/1/calculateRoute/{route_coords}/json'
    headers = {'Content-Type': 'application/json'}
    payload = {
        "key": API_KEY,
        "traffic": True,
        "departAt": depart_at_utc,
        "computeTravelTimeFor": "all"
    }

    try:
        response = requests.post(url, headers=headers, json=payload)
 
        if response.status_code == 200:
            data = response.json()
            if 'routes' in data and data['routes']:
                summary = data['routes'][0]['summary']
                no_traffic = summary.get('noTrafficTravelTimeInSeconds', 0)
                traffic_delay = summary.get('trafficDelayInSeconds', 0)
                if no_traffic > 0:
                    return (no_traffic + traffic_delay) / no_traffic
    except Exception as e:
        print(f"API fetch error for {depart_at_utc} on {route_coords}: {e}")
    return None

 
def generate_recent_times(shift_type, num_samples=10):
    now = datetime(2024, 10, 11)  # Base on real data availability
    times = []
    delta_map = {
        'weekday_8am': (now.weekday() + 7, '%Y-%m-%dT19:00:00Z'),  # Prev day 19:00 UTC
        'weekday_13pm': (now.weekday() + 7 - 5, '%Y-%m-%dT00:00:00Z'),  # Same day 00:00 UTC
        'saturday_8am': ((now.weekday() - 5) % 7, '%Y-%m-%dT19:00:00Z'),
        'saturday_13pm': ((now.weekday() - 5) % 7, '%Y-%m-%dT00:00:00Z')
    }
    base_days, fmt = delta_map[shift_type]
    for i in range(num_samples):
        target_date = now - timedelta(days=base_days + 7 * i)
        times.append(target_date.strftime(fmt))
    return times

# Fetch and compute params (average over 3 routes)
def compute_scenario_params(shift_type):
    utc_times = generate_recent_times(shift_type)
    all_mults = []
    for dt in utc_times:
        route_mults = [get_multiplier(dt, route[0]) for route in routes]
        route_mults = [m for m in route_mults if m is not None]
        if route_mults:
            all_mults.append(np.mean(route_mults))  
    all_mults = [m for m in all_mults if m is not None]
    if len(all_mults) >= 3:  
        mean_mult = np.mean(all_mults)
        cv = np.std(all_mults) / mean_mult if mean_mult > 0 else 0.15
   
        if cv < 0.05:
            cv = 0.10 + cv   
    else:
       
        fallback = {
            'weekday_8am': (1.42, 0.20),  # 42% congestion
            'weekday_13pm': (1.15, 0.15),  # 15% congestion
            'saturday_8am': (1.02, 0.10),  # 2% congestion
            'saturday_13pm': (1.23, 0.15)  # 23% congestion
        }
        mean_mult, cv = fallback.get(shift_type, (1.0, 0.15))
    print(f"{shift_type}: Fetched {len(all_mults)} samples (avg over routes), Mean: {mean_mult:.2f}x, CV: {cv:.2f}")
    return mean_mult, cv

from datetime import timezone

now_utc = datetime.now(timezone.utc).strftime('%Y-%m-%dT%H:%M:%SZ')
live_mults = [get_multiplier(now_utc, route[0]) for route in routes]

live_mults = [m for m in live_mults if m is not None]
real_time_mult = np.mean(live_mults) if live_mults else None
print(f"Live real-time mult (Sat 1 PM, avg routes): {real_time_mult:.3f}x" if real_time_mult else "Live fetch failed")

 
mean_wd8, cv_wd8 = compute_scenario_params('weekday_8am')
mean_wd13, cv_wd13 = compute_scenario_params('weekday_13pm')
mean_sat8, cv_sat8 = compute_scenario_params('saturday_8am')
mean_sat13, cv_sat13 = compute_scenario_params('saturday_13pm')
 
if real_time_mult:
 
    sat13_mults = [real_time_mult]   
  
    mean_sat13 = real_time_mult
    cv_sat13 = 0.15  

TRAFFIC_PARAMS = {
    ('weekday', 8): (mean_wd8, cv_wd8),
    ('weekday', 13): (mean_wd13, cv_wd13),
    ('saturday', 8): (mean_sat8, cv_sat8),
    ('saturday', 13): (mean_sat13, cv_sat13)
}

 
def get_lognorm_params(mean_mult, cv):
    sigma = np.sqrt(np.log(1 + cv**2))
    mu = np.log(mean_mult) - 0.5 * sigma**2
    return sigma, np.exp(mu)

LOGNORM_PARAMS = {}
for (day, hour), (mean_mult, cv) in TRAFFIC_PARAMS.items():
    sigma, scale = get_lognorm_params(mean_mult, cv)
    LOGNORM_PARAMS[(day, hour)] = (sigma, scale)
    print(f"{day.capitalize()} {hour}:00 - Mean: {mean_mult:.2f}x, CV: {cv:.2f}, Lognorm(s={sigma:.3f}, scale={scale:.3f})")
 
def sample_route_multipliers(day_type, shift_hour, num_legs, correlated=False):
    sigma, scale = LOGNORM_PARAMS[(day_type, shift_hour)]
    samples = lognorm.rvs(s=sigma, scale=scale, size=1 if correlated else num_legs)
    return np.full(num_legs, samples[0]) if correlated else samples

# Example
np.random.seed(42)
example_base = TravelDF.loc['CentrePort Wellington', 'Woolworths Newtown']
mult = sample_route_multipliers('saturday', 13, 1)[0]
sim_time = example_base * mult
print(f"Example Sat 1 PM: Base {example_base / 60:.1f} min -> Sim {sim_time / 60:.1f} min (mult={mult:.2f}x)")
 

Live fetch failed
weekday_8am: Fetched 0 samples (avg over routes), Mean: 1.42x, CV: 0.20
weekday_13pm: Fetched 0 samples (avg over routes), Mean: 1.15x, CV: 0.15
saturday_8am: Fetched 0 samples (avg over routes), Mean: 1.02x, CV: 0.10
saturday_13pm: Fetched 0 samples (avg over routes), Mean: 1.23x, CV: 0.15
Weekday 8:00 - Mean: 1.42x, CV: 0.20, Lognorm(s=0.198, scale=1.392)
Weekday 13:00 - Mean: 1.15x, CV: 0.15, Lognorm(s=0.149, scale=1.137)
Saturday 8:00 - Mean: 1.02x, CV: 0.10, Lognorm(s=0.100, scale=1.015)
Saturday 13:00 - Mean: 1.23x, CV: 0.15, Lognorm(s=0.149, scale=1.216)
Example Sat 1 PM: Base 9.7 min -> Sim 12.6 min (mult=1.31x)


In [ ]:
#the hardcoded test one without api calls 
import pandas as pd
from scipy.stats import lognorm

# Scenarios: (day_type, shift_hour) -> (mean_multiplier, cv)
TRAFFIC_PARAMS = {
    ('weekday', 8):  (1.60, 0.20),   # Morning peak
    ('weekday', 13): (1.38, 0.15),   # Midday
    ('saturday', 8): (1.09, 0.10),   # Morning low
    ('saturday', 13): (1.53, 0.15)   # Midday moderate
}

def get_lognorm_params(mean_mult, cv):
    sigma = np.sqrt(np.log(1 + cv**2))
    mu = np.log(mean_mult) - 0.5 * sigma**2
    return sigma, np.exp(mu)

LOGNORM_PARAMS = {}
for (day, hour), (mean_mult, cv) in TRAFFIC_PARAMS.items():
    sigma, scale = get_lognorm_params(mean_mult, cv)
    LOGNORM_PARAMS[(day, hour)] = (sigma, scale)
    print(f"{day.capitalize()} {hour}:00 shift - Mean mult: {mean_mult:.2f}x, CV: {cv:.2f}, "
          f"Lognorm(s={sigma:.3f}, scale={scale:.3f})")

def sample_route_multipliers(day_type, shift_hour, num_legs, correlated=False):
    sigma, scale = LOGNORM_PARAMS[(day_type, shift_hour)]
    if correlated:
        return np.full(num_legs, lognorm.rvs(s=sigma, scale=scale, size=1))
    return lognorm.rvs(s=sigma, scale=scale, size=num_legs)

# Example
np.random.seed(42)  # For reproducibility
example_base = TravelDF.loc['CentrePort Wellington', 'Woolworths Newtown']
mult = sample_route_multipliers('weekday', 8, 1)[0]
sim_time = example_base * mult
print(f"Example: Base {example_base / 60:.1f} min -> Sim {sim_time / 60:.1f} min (mult={mult:.2f}x)") 

Weekday 8:00 shift - Mean mult: 1.60x, CV: 0.20, Lognorm(s=0.198, scale=1.569)
Weekday 13:00 shift - Mean mult: 1.38x, CV: 0.15, Lognorm(s=0.149, scale=1.365)
Saturday 8:00 shift - Mean mult: 1.09x, CV: 0.10, Lognorm(s=0.100, scale=1.085)
Saturday 13:00 shift - Mean mult: 1.53x, CV: 0.15, Lognorm(s=0.149, scale=1.513)
Example: Base 9.7 min -> Sim 16.7 min (mult=1.73x)


#### Initialise Route Class

In [ ]:
class Route():
  def __init__(self, travelDF, demandDF, saturday = False,start = "CentrePort Wellington"):
    self.travelDF = travelDF
    self.demandDF = demandDF
    self.saturday = saturday
    self.route = [start]
    self.start = start
    self.travelTime = 0
    self.demand = 0
    self.vanCost = 0
    self.subCost = 0



  def __repr__(self):
    return (f"Route: {str(self.route)},\nTravel Time = {self.travelTime} seconds,\nTotal Demand = {self.demand}")

  def __eq__(self, other):
    if not isinstance(other, Route):
        return NotImplemented
    # Compare routes based on the set of stores, ignoring order
    return set(self.route[1:-1]) == set(other.route[1:-1])

  def addStop(self, stop, index=np.inf):
    original_route_length = len(self.route)
    if index > original_route_length:
      index = original_route_length

    #Travel Times
    if index == 0:
        self.travelTime += self.travelDF.loc[stop, self.route[0]]

    elif index == original_route_length:
        self.travelTime += self.travelDF.loc[self.route[-1], stop]

    else:
        self.travelTime -= self.travelDF.loc[self.route[index-1], self.route[index]]
        self.travelTime += self.travelDF.loc[self.route[index-1], stop]
        self.travelTime += self.travelDF.loc[stop, self.route[index]]

    #Demand
    if stop != 'CentrePort Wellington':
      if not self.saturday:
        self.demand += self.demandDF.loc[stop].iloc[0]
      else:
        self.demand += self.demandDF.loc[stop].iloc[1]

    #Route
    self.route.insert(index, stop)


    if not self.saturday:
        self.travelTime += 900 * self.demandDF.loc[stop].iloc[0]
    else:
        self.travelTime += 900 * self.demandDF.loc[stop].iloc[1]

  def deleteStop(self, index):
    if index > len(self.route):
      index = len(self.route)

    if self.route[index] != "CentrePort Wellington":
      self.travelTime -= 900

    if not self.saturday:
      self.demand -= self.demandDF.loc[self.route[index]].iloc[0]
    else:
      self.demand -= self.demandDF.loc[self.route[index]].iloc[1]

    if index == 0:
        self.travelTime -= self.travelDF.loc[self.route[0], self.route[1]]
    elif index == len(self.route):
        self.travelTime -= self.travelDF.loc[self.route[-2], self.route[-1]]
    else:
        self.travelTime -= self.travelDF.loc[self.route[index], self.route[index+1]]
        self.travelTime -= self.travelDF.loc[self.route[index-1], self.route[index]]
        self.travelTime += self.travelDF.loc[self.route[index-1], self.route[index+1]]

    self.route.pop(index)



  def complete_round_trip(self):
      if self.route[-1] != self.start:
          self.route.append(self.start)
          self.travelTime += self.travelDF.loc[self.route[-2], self.start]

  def addList(self,list):
    for store in list:
      self.addStop(store)

  def isInRoute(self, stop):
    for store in self.route:
      if store == stop:
        return True
    return False

  def testDemand(self, store):
    if store != 'CentrePort Wellington':
      if not self.saturday:
        return self.demand + self.demandDF.loc[store].iloc[0]
      else:
        return self.demand + self.demandDF.loc[store].iloc[1]
    return self.demand

  def testTime(self, stop, index=np.inf):
    original_route_length = len(self.route)
    if index > original_route_length:
      index = original_route_length

    #Travel Times
    if index == 0:
        return self.travelTime + self.travelDF.loc[stop, self.route[0]]

    elif index == original_route_length:
        return self.travelTime + self.travelDF.loc[self.route[-1], stop]

    else:
        time = self.travelTime
        time -= self.travelDF.loc[self.route[index-1], self.route[index]]
        time += self.travelDF.loc[self.route[index-1], stop]
        time += self.travelDF.loc[stop, self.route[index]]
        return time

  def genCost(self):
    overtime = max(0, (self.travelTime-3*3600))
    subTime = np.ceil(self.travelTime/(4*3600))
    self.vanCost = (((200/3600)*self.travelTime)+(75/3600)*overtime)
    self.subCost = 1000*subTime


#### Cheapest Insertion Routes:
Insertion until total demand surpasses 9, or total time exceeds 10800

In [ ]:
def cheapest_insertion_route(travelDF, demandDF, remaining_stores = None,is_saturday=False, max_boxes=9, max_time=10800):
    route = Route(travelDF, demandDF, saturday=is_saturday)
    route.route = ["CentrePort Wellington", "CentrePort Wellington"]
    route.travelTime = 0
    route.demand = 0
    if remaining_stores is None:
        remaining_stores = set(demandDF.index)

    stores_added_in_iteration = True
    while stores_added_in_iteration:
        stores_added_in_iteration = False
        best_store = None
        best_position = None
        best_time_increase = float('inf')

        for store in remaining_stores:
            store_demand = demandDF.loc[store]['Saturday' if is_saturday else 'Weekday']
            if route.demand + store_demand > max_boxes:
                continue

            for pos in range(1, len(route.route)):
                prev_stop = route.route[pos - 1]
                next_stop = route.route[pos]

                added_time = (
                    travelDF.loc[prev_stop, store] +
                    travelDF.loc[store, next_stop] -
                    travelDF.loc[prev_stop, next_stop]
                )

                if (route.travelTime + added_time) > max_time:
                    continue

                if added_time < best_time_increase:
                    best_store = store
                    best_position = pos
                    best_time_increase = added_time

        if best_store is not None:
            route.addStop(best_store, best_position)
            remaining_stores.remove(best_store)
            stores_added_in_iteration = True

    if len(route.route) > 2: # Only return route if stores were added
        return route
    else:
        return False # Return False if no stores could be added to this route

### Route Generation


In [ ]:
def generate_all_feasible_routes(travelDF, demandDF, saturday=False,
                                 max_boxes=9, max_time=10800,
                                 current_route=["CentrePort Wellington"],
                                 all_routes=None, min_return_time=None, demand_dict=None):
    if all_routes is None:
        all_routes = []

    if min_return_time is None:
        depot = "CentrePort Wellington"
        min_return_time = {store: travelDF.loc[store, depot] for store in demandDF.index}
    if demand_dict is None:
        demand_dict = {store: (row.iloc[0], row.iloc[1]) for store, row in demandDF.iterrows()}

    end_node = current_route[-1]

    completed_route = Route(travelDF, demandDF, saturday=saturday)
    completed_route.addList(current_route[1:])


    if end_node != "CentrePort Wellington":
        completed_route.complete_round_trip()
        completed_route.genCost()
        if completed_route.travelTime <= max_time:
            if completed_route not in all_routes:

                all_routes.append(completed_route)
            else:
                for i, r in enumerate(all_routes):
                    if r == completed_route:
                        if completed_route.travelTime < r.travelTime:
                            all_routes[i] = completed_route
                        break

    new_route = Route(travelDF, demandDF, saturday=saturday)
    new_route.addList(current_route[1:])

    for store in demandDF.index:
        if store not in new_route.route:
            if saturday and demand_dict[store][1] == 0:
                continue

            new_demand = new_route.testDemand(store)
            if new_demand <= max_boxes:
                new_time = new_route.testTime(store)

                if new_time + min(min_return_time.values()) > max_time:
                    continue

                if new_time <= max_time:
                    generate_all_feasible_routes(
                        travelDF, demandDF, saturday, max_boxes, max_time,
                        current_route + [store], all_routes,
                        min_return_time=min_return_time,
                        demand_dict=demand_dict
                    )

    return all_routes


In [ ]:
all_weekday_routes = generate_all_feasible_routes(TravelDF, demand75, max_time=(10*3600))
print(f"found {len(all_weekday_routes)} routes")


found 598 routes


In [ ]:
maxDemand = max(all_weekday_routes, key=lambda route: route.demand)
print(f"The route with the maximum demand on a weekday is: {maxDemand.demand}")
maxTime = max(all_weekday_routes, key=lambda route: route.travelTime)
print(f"The route with the maximum time on a weekday is: {maxTime.travelTime}")

The route with the maximum demand on a weekday is: 8.997535082965873
The route with the maximum time on a weekday is: 15393.753337568616


In [ ]:
all_saturday_routes = generate_all_feasible_routes(TravelDF, demand75, True)
print(f"found {len(all_saturday_routes)} routes")

found 3040 routes


In [ ]:
maxsatDemand = max(all_saturday_routes, key=lambda route: route.demand)
print(f"The route with the maximum demand on a saturday is: {maxsatDemand.demand}")
maxsatTime = max(all_saturday_routes, key=lambda route: route.travelTime)
print(f"The route with the maximum time on a saturday is: {maxsatTime.travelTime}")

The route with the maximum demand on a saturday is: 6.4153975928445215
The route with the maximum time on a saturday is: 10799.480577065395


In [ ]:
print(f"found {len(all_weekday_routes)} feasible routes on the weekday, and {len(all_saturday_routes)} on the saturday.")

found 598 feasible routes on the weekday, and 3040 on the saturday.


In [ ]:
demand75

,Weekday,Saturday
Store,,
FreshChoice Cannons Creek,2.587546,0.000000
FreshChoice Cuba Street,2.246321,0.000000
FreshChoice Woburn,2.194569,0.000000
Metro Cable Car Lane,2.587546,0.000000
Woolworths Aotea,3.034719,1.175000
Woolworths Crofton Downs,3.675902,0.587500
Woolworths Johnsonville,3.852079,0.587500
Woolworths Johnsonville Mall,3.035806,1.396263
Woolworths Karori,2.908907,0.587500


## Linear Programming

In [ ]:
%pip install PuLP

Note: you may need to restart the kernel to use updated packages.


### Linear Program

In [ ]:
from pulp import *
import gurobipy

In [ ]:
prob = LpProblem("Total_Cost_For_Woolworths", LpMinimize)

In [ ]:
# Generate Weekday Routes
weekday_routes = {f"W_Route{m}": all_weekday_routes[m] for m in range(len(all_weekday_routes))}

# Generate Saturday Routes
saturday_routes = {f"S_Route{m}": all_saturday_routes[m] for m in range(len(all_saturday_routes))}

# Combine all routes
all_routes = {**weekday_routes, **saturday_routes}
all_routes_keys = list(all_routes.keys())

# Decision variables
Selected = LpVariable.dicts("Selected", all_routes_keys, 0, 1, LpBinary)
Van = LpVariable.dicts("Van", all_routes_keys, 0, 1, LpBinary)
Overtime = LpVariable.dicts("Overtime", all_routes_keys, 0, None, LpContinuous)
SubCost = LpVariable.dicts("SubCost", all_routes_keys, 0, None, LpInteger)
Z = LpVariable.dicts("Z", all_routes_keys, 0, 1, LpBinary)  # Selected * Van
W = LpVariable.dicts("W", all_routes_keys, 0, None, LpContinuous)  # Selected * SubCost
Y = LpVariable.dicts("Y", all_routes_keys, 0, 1, LpBinary)
Fleet = LpVariable("Fleet", 0, 4, LpInteger)

In [ ]:
# Create route coverage dictionary
stores_to_cover = demand75.index.tolist()

# Combined
AllRouteDict = {}

for store in stores_to_cover:
    AllRouteDict[store] = {}
    for route_key in all_routes_keys:
        actual_stores_in_route = all_routes[route_key].route
        if store in actual_stores_in_route:
            AllRouteDict[store][route_key] = 1
        else:
            AllRouteDict[store][route_key] = 0

# Saturday only coverage
sat_stores_to_cover = [
    store for store in stores_to_cover
    if demand75.loc[store, 'Saturday'] != 0
]

In [ ]:
# Route data
AllTime = pd.Series({key: route.travelTime for key, route in all_routes.items()})
AllDemandWeekday = pd.Series({key: route.demand for key, route in weekday_routes.items()})
AllDemandSaturday = pd.Series({key: route.demand for key, route in saturday_routes.items()})


AllRouteData = pd.DataFrame({
    'TravelTime': AllTime,
    'DemandWeekday': AllDemandWeekday,
    'DemandSaturday': AllDemandSaturday,
})

In [ ]:
AllRouteData = pd.DataFrame({
    'TravelTime': {key: route.travelTime for key, route in all_routes.items()},
    'Demand': {key: route.demand for key, route in all_routes.items()},
    'vanCost': {key: route.vanCost for key, route in all_routes.items()},
    'subCost': {key: route.subCost for key, route in all_routes.items()},
})

In [ ]:
prob = LpProblem("Total_Cost_For_Woolworths", LpMinimize)

prob += (
    lpSum(
        Z[i] * AllRouteData.loc[i, 'vanCost']
        + (Selected[i] - Z[i]) * AllRouteData.loc[i, 'subCost']
        for i in all_routes_keys
    )
    + 100000 / ((6 / 7) * 365) * Fleet
)


# Linearize z[i] = Selected[i] & Van[i]
for i in all_routes_keys:
    prob += Z[i] <= Selected[i]
    prob += Z[i] <= Van[i]
    prob += Z[i] >= Selected[i] + Van[i] - 1

# Demand constraint
BigDemand = 15
for i in all_routes.keys():
    prob += AllRouteData.loc[i,'Demand'] - 5*Van[i] <= 4 + BigDemand*(1 - Selected[i])


# Van / Fleet constraint
# This constraint needs to be split for weekday and saturday selections
prob += lpSum(Z[i] for i in weekday_routes.keys()) <= 2 * Fleet
prob += lpSum(Z[i] for i in saturday_routes.keys()) <= 2 * Fleet


# Partitioning constraint (link routes to selected)
# This constraint needs to be split for weekday and saturday selections
for store in stores_to_cover:
    prob += lpSum(AllRouteDict[store][j] * Selected[j] for j in weekday_routes.keys()) == 1

for store in sat_stores_to_cover:
     prob += lpSum(AllRouteDict[store][j] * Selected[j] for j in saturday_routes.keys()) == 1


# Write LP file and solve
prob.writeLP("Woolworthsweekdayproblem.lp")

#solver = GUROBI_CMD(msg=True, timeLimit=1800)  # timeLimit optional
solver = HiGHS_CMD(msg=True, timeLimit=1800, threads=8)
#prob.solve(solver)
prob.solve()

finalRoutes = []
finalWeek = []
finalSat = []

# Status
print("Status:", LpStatus[prob.status])

print(f"Fleet: {Fleet.varValue}")

# Print selected routes with stores, van/subcontractor info, and subcontractor cost
print("Selected Weekday routes and details:")
for i in weekday_routes.keys():
    if Selected[i].varValue >= 1:
        stores_in_route = all_routes[i].route
        print(f"{i} -> Stores: {stores_in_route}")
        finalRoutes.append(weekday_routes[i])
        finalWeek.append(weekday_routes[i])

print("Selected Saturday routes and details:")
for i in saturday_routes.keys():
    if Selected[i].varValue >= 1:
        stores_in_route = all_routes[i].route
        print(f"{i} -> Stores: {stores_in_route}")
        finalRoutes.append(saturday_routes[i])
        finalSat.append(saturday_routes[i])

# Optimised objective
print("Total Cost = ", value(prob.objective))

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/jackmaara/anaconda3/envs/myenv/lib/python3.13/site-packages/pulp/apis/../solverdir/cbc/osx/i64/cbc /var/folders/96/9kz9fjl12mv9rc28jy8p6jzw0000gn/T/70667661c0d5496e863895c7b3b0d355-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /var/folders/96/9kz9fjl12mv9rc28jy8p6jzw0000gn/T/70667661c0d5496e863895c7b3b0d355-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 14592 COLUMNS
At line 95636 RHS
At line 110224 BOUNDS
At line 121140 ENDATA
Problem MODEL has 14587 rows, 10915 columns and 51936 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 6675.55 - 0.13 seconds
Cgl0003I 0 fixed, 0 tightened bounds, 2124 strengthened rows, 3266 substitutions
Cgl0004I processed model has 6636 rows, 7680 columns (7680 integer (7679 of which binary)) and 49921 elements
Cbc0038I Initial state - 

In [ ]:
finalRoutes[0]

Route: ['CentrePort Wellington', 'Woolworths Aotea', 'FreshChoice Cannons Creek', 'Woolworths Tawa', 'CentrePort Wellington'],
Travel Time = 10497.456578567324 seconds,
Total Demand = 8.171018420630357

In [ ]:
import folium
import openrouteservice as ors

In [ ]:
ORSkey = "REMOVED"

In [ ]:
locations = pd.read_csv("WoolworthsLocations.csv")

store_names = locations['Store'].tolist()
coords = locations[['Long', 'Lat']] # Mapping packages work with Long, Lat arrays
coords = coords.to_numpy().tolist() # Make the arrays into a list of lists.



# Folium, however, requires Lat, Long arrays - so a reversal is needed.
m = folium.Map(location = list(reversed(coords[2])), zoom_start=10)

# NOT RUN, to plot first store.
# folium.Marker(list(reversed(coords[0])), popup = locations.Store[0], icon = folium.Icon(color = 'black')).add_to(m)

for i in range(0, len(coords)):
    if locations.Type[i] == "Woolworths":
        iconCol = "green"
    elif locations.Type[i] == "CentrePort":
        iconCol = "black"
    elif locations.Type[i] == "Metro":
        iconCol = "orange"
    elif locations.Type[i] == "FreshChoice":
        iconCol = "blue"
    folium.Marker(list(reversed(coords[i])), popup = locations.Store[i], icon = folium.Icon(color = iconCol)).add_to(m)

store_names
store_index = {name: i for i, name in enumerate(store_names)}

client = ors.Client(key=ORSkey)

weekMapRoutes = []
for route in finalWeek:
    routeCoords = []
    for store in route.route:
        routeCoords.append(coords[store_index[store]])

    weekMapRoutes.append(client.directions(
        coordinates = routeCoords, # Distribution Centre to Woolworths Newtown
        # We can have more than two coords - it will generate a path between those coords in order.
        profile = 'driving-hgv', # can be driving-car, driving-hgv, etc.
        format='geojson',
        validate = False
        ))

satMapRoutes = []
for route in finalSat:
    routeCoords = []
    for store in route.route:
        routeCoords.append(coords[store_index[store]])

    satMapRoutes.append(client.directions(
        coordinates=routeCoords,
        profile='driving-hgv',
        format='geojson',
        validate=False
    ))



In [ ]:
for route in weekMapRoutes:
    for feature in route['features']:
        folium.PolyLine(
            locations=[list(reversed(coord)) for coord in feature['geometry']['coordinates']],
            color='red',
            weight = 5,
            opacity = 0.8
        ).add_to(m)

m

In [ ]:
m = folium.Map(location = list(reversed(coords[2])), zoom_start=10)

for i in range(0, len(coords)):
    if locations.Type[i] == "Woolworths":
        iconCol = "green"
    elif locations.Type[i] == "CentrePort":
        iconCol = "black"
    else:
        continue
    folium.Marker(list(reversed(coords[i])), popup = locations.Store[i], icon = folium.Icon(color = iconCol)).add_to(m)

for route in satMapRoutes:
    for feature in route['features']:
        folium.PolyLine(
            locations=[list(reversed(coord)) for coord in feature['geometry']['coordinates']],
            color='green',
            weight = 5,
            opacity = 0.8
        ).add_to(m)

m